In [2]:
%matplotlib inline
from mytools3 import read_part_wetzel
import yt
import numpy as np
import matplotlib.pyplot as plt

In [11]:
import matplotlib
from mytools3 import read_part_wetzel,backup_file,isint,isfloat,boxmsk
from lg_scripts.hydro_scripts.dwarfs.constants import center_position_dict
import sys,yt
from yt.units import kiloparsec, Msun, km, s
# from optparse import OptionParser
import argparse
import wutilities as ut
import rockstar
import numpy as np



def parse_type_ops(opsstring,def_nref):
    ops = opsstring.split(',')
    opsdict = {}
    n_ref = None
    for op in ops:
        key,val = op.split(':')
        key = key.strip()
        val = val.strip()
        if key == 'n_ref':
            n_ref = int(val)
            continue

        if isint(val):
            opsdict[key] = int(val)
        elif isfloat(val):
            opsdict[key] = float(val)
        else:
            opsdict[key] = val

    if def_nref > 0:
        n_ref = def_nref

    if n_ref is None:
        raise KeyError("Must specify either a default n_ref or pass it in as part of the options for each particle type.")
    return opsdict,n_ref


def parse_img_rotate(rotation, axis):
    assert axis in [0, 1, 2, 'x', 'y', 'z']
    rename = {'x':0, 'y':1, 'z':2}
    if axis in rename:  axis = rename[axis]
    
    assert rotation in [0, 90, 180, 270]
    
    #easy case first -- no rotation, no inversions
    if rotation == 0:
        return [0, 1, 2], [1, 1, 1]

    if axis == 0:   #projecting along x axis => z and y are plotted by default, with y on the x-axis and z on the y-axis
        if rotation == 90:   #then I want -z on the y axis and +y on the x-axis
            return [0, 2, 1], [1, 1, -1]
        elif rotation == 180:  #effectively flip both axes
            return [0, 1, 2], [1, -1, -1]
        elif rotation == 270: 
            return [0, 2, 1], [1, -1, 1]
    elif axis == 1:  #projecting along y axis => z and x are plotted by default, with x on the y-axis and z on the y-axis
        if rotation == 90:  #then I want -z on the y axis and +x on the x axis => z -> -x, x -> z
            return [2, 1, 0], [-1, 1, 1]
        elif rotation == 180:
            return [0, 1, 2], [-1, 1, -1]
        elif rotation == 270:
            return [2, 1, 0], [1, 1, -1]
    elif axis == 2:     #prjecting along z axis => x on x-axis, y on y-axis
        if rotation == 90:
            return [1, 0, 2], [1, -1, 1]
        elif rotation == 180:
            return [0, 1, 2], [-1, -1, 0]
        elif rotation == 270:
            return [1, 0, 2], [-1, 1, 1]
    raise KeyError("Passed in an invalid rotation or axis somehow")

In [30]:
part = read_part_wetzel('output/snapdir_600/', ['star', 'gas'], properties=['position','velocity','mass'], assign_center=True)


# in wutilities.simulation.Snapshot():
  read snapshot_times.txt
  reading snapshot index = 600, redshift = 0.000

# in gizmo_analysis.gizmo_io.Read():
* read header from: output/snapdir_600/snapshot_600.0.hdf5
  snapshot contains the following number of particles:
  dark      (id = 1): 70514272 particles
  dark.2    (id = 2): 5513331 particles
  gas       (id = 0): 57060074 particles
  star      (id = 4): 13976485 particles
  blackhole (id = 5): 0 particles

* read particles
  read star  : ['mass', 'position', 'velocity']
  read gas   : ['mass', 'position', 'velocity']
  from: snapshot_600.0.hdf5
  from: snapshot_600.1.hdf5
  from: snapshot_600.2.hdf5
  from: snapshot_600.3.hdf5

* read cosmological parameters from: initial_conditions/ic_agora_m12i.conf

* checking sanity of particle properties

* assigning center of galaxy/halo:
  position = (41792.147, 44131.235, 46267.679) [kpc comoving]
  velocity = (-52.5, 71.9, 95.2) [km / s]



In [8]:
cen = part.center_position
vel = part.center_velocity

In [44]:
bsize = 30
bl = [-bsize/2, bsize/2]
bbox = np.array( [ bl, bl, bl])

data = {}
for ptype in part:
    msk = boxmsk(part[ptype]['position'], cen, bsize/2)

    data[(ptype,'particle_mass')] = part[ptype]['mass'][msk]

    data[(ptype,'particle_position_x')] = part[ptype]['position'][msk][:,0] - cen[0]
    data[(ptype,'particle_position_y')] = part[ptype]['position'][msk][:,1] - cen[1]
    data[(ptype,'particle_position_z')] = part[ptype]['position'][msk][:,2] - cen[2]

    data[(ptype,'particle_velocity_x')] = part[ptype]['velocity'][msk][:,0] - vel[0]
    data[(ptype,'particle_velocity_y')] = part[ptype]['velocity'][msk][:,1] - vel[1]
    data[(ptype,'particle_velocity_z')] = part[ptype]['velocity'][msk][:,2] - vel[2]


ds = yt.load_particles(data, length_unit=kiloparsec, mass_unit=Msun, n_ref=32, bbox=bbox, velocity_unit=km/s)


yt : [INFO     ] 2018-05-25 14:46:30,928 Parameters: current_time              = 0.0
yt : [INFO     ] 2018-05-25 14:46:30,929 Parameters: domain_dimensions         = [2 2 2]
yt : [INFO     ] 2018-05-25 14:46:30,930 Parameters: domain_left_edge          = [-15. -15. -15.]
yt : [INFO     ] 2018-05-25 14:46:30,931 Parameters: domain_right_edge         = [15. 15. 15.]
yt : [INFO     ] 2018-05-25 14:46:30,931 Parameters: cosmological_simulation   = 0.0


In [49]:
prj = yt.ProjectionPlot(ds, 0, ('deposit', 'gas_nn_velocity_x'), center=[0,0,0], width=(bsize, 'kpc'))

yt : [INFO     ] 2018-05-25 14:48:02,641 Projection completed
yt : [INFO     ] 2018-05-25 14:48:02,641 xlim = -15.000000 15.000000
yt : [INFO     ] 2018-05-25 14:48:02,642 ylim = -15.000000 15.000000
yt : [INFO     ] 2018-05-25 14:48:02,643 xlim = -15.000000 15.000000
yt : [INFO     ] 2018-05-25 14:48:02,643 ylim = -15.000000 15.000000
yt : [INFO     ] 2018-05-25 14:48:02,644 Making a fixed resolution buffer of (('deposit', 'gas_nn_velocity_x')) 800 by 800


In [50]:
# prj.set_unit('gas_cic_velocity_x','km/s')
prj.save()

yt : [INFO     ] 2018-05-25 14:48:04,006 Saving plot ParticleData_Projection_x_gas_nn_velocity_x.png


['ParticleData_Projection_x_gas_nn_velocity_x.png']